In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import datetime as dt
import sqlite3 as db
import matplotlib.pyplot as pp

In [2]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [3]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [33]:
def weeklyAbundance():
    cnx = connectDB()
    try:
        ObsDataset = pd.read_sql('SELECT comName,locId,obsDt,howMany,subId FROM historicObservations WHERE comName in ("Canada Jay","Orange-crowned Warbler");',con=cnx,parse_dates=['obsDt']) #Canada Jay filter is for testing purposes only
        ObsDataset['obsDt_week'] = ObsDataset['obsDt'].dt.isocalendar().week
        #group up data
        ObsDataset = ObsDataset.groupby(['comName','locId','obsDt_week'])['howMany'].median()
        ObsDataset = pd.DataFrame(ObsDataset)
        ObsDataset['relativeAbundance'] = (ObsDataset['howMany']/ObsDataset['howMany'].mean())/max(ObsDataset['howMany']/ObsDataset['howMany'].mean())
        ObsDataset.sort_values(by=['comName','obsDt_week'],ascending=True,inplace=True)   #not needed
        ObsDataset.reset_index(inplace=True)   #not needed
    except Exception as calcEx:
        raise UserWarning(calcEx)
    finally: cnx.close()
    return ObsDataset

In [34]:
trx = weeklyAbundance()
trx

,comName,locId,obsDt_week,howMany,relativeAbundance
0,Canada Jay,L8312096,1,1.0,0.076923
1,Canada Jay,L12172834,10,6.0,0.461538
2,Canada Jay,L12172834,11,2.0,0.153846
3,Canada Jay,L10129014,13,4.0,0.307692
4,Canada Jay,L12172834,40,7.0,0.538462
5,Canada Jay,L12172834,49,2.0,0.153846
6,Canada Jay,L8312096,49,1.0,0.076923
7,Canada Jay,L12172834,52,4.0,0.307692
8,Canada Jay,L424117,52,3.0,0.230769
9,Canada Jay,L8312096,52,1.0,0.076923


In [41]:
##TODO #67 interpolate sighting values into the test Canada Jay dataset

In [ ]:
##TODO #66 split out birds into quartiles based on the number of observations available and the YOY frequency. Results will determine the strength of forecasting required

In [39]:
##TODO #64 using the relative abundance value from weeklyAbundance(), write a function that returns FAO land coverage preference
##TODO #68 add seasons to relative abundance function, then add in a loop over each season in the function seasonalHabitatPreference
def seasonalHabitatPref():
    cnx = connectDB()
    try:
        wklyAbd = weeklyAbundance()
        #start season loop here
        listy = []
        for locId in wklyAbd.itertuples():
            query = f'SELECT Barren,Urban,Grasslands,Savannas,waterBodies,evergreenNeedleleafForest,evergreenBroadleafForests,openForests,mixedBroadleafandNeedleleafForests,sparseForests,denseHerbaceous,sparseHerbaceous FROM FAO_by_locId WHERE FAO_by_locId.locId = "{locId.locId}"'
            lx = pd.read_sql(query,con=cnx)
            lx = lx.applymap(lambda x: locId.relativeAbundance*x,na_action='ignore')
            lx['comName'] = locId.comName
            lx['week'] = locId.obsDt_week
            listy.append(lx)
        results = pd.DataFrame()
        results = pd.concat(listy,ignore_index=True)
        results = results.groupby(['comName','week']).mean()
    except Exception as prefExc:
        raise UserWarning(prefExc)
    finally: cnx.close()
    return results

In [40]:
adun = seasonalHabitatPref()
adun


Urban  Savannas  evergreenNeedleleafForest  \
comName                week                                               
Canada Jay             1       NaN       NaN                   0.010000   
                       10      NaN       NaN                   0.461538   
                       11      NaN       NaN                   0.153846   
                       13     0.24       NaN                   0.067692   
                       40      NaN       NaN                   0.538462   
                       49      NaN       NaN                   0.081923   
                       52      NaN       NaN                   0.117436   
                       53      NaN       NaN                   0.615385   
Orange-crowned Warbler 16      NaN       NaN                   0.057692   
                       17      NaN       NaN                   0.035769   
                       18      NaN  0.016923                   0.047500   
                       19      NaN       NaN                   0.099744   
                       20      NaN       NaN                   0.010000   
                       21      NaN       NaN                   0.153846   
                       22      NaN  0.016923                   0.025385   
                       24      NaN       NaN                   0.033846   
                       26      NaN       NaN                   0.111923   
                       29      NaN       NaN                   0.115385   
                       36      NaN       NaN                   0.090000   
                       39      NaN       NaN                   0.010000   
                       42      NaN       NaN                        NaN   

                             evergreenBroadleafForests  openForests  \
comName                week                                           
Canada Jay             1                           NaN          NaN   
                       10                          NaN          NaN   
                       11                          NaN          NaN   
                       13                          NaN     0.058462   
                       40                          NaN          NaN   
                       49                          NaN          NaN   
                       52                     0.030000          NaN   
                       53                          NaN          NaN   
Orange-crowned Warbler 16                          NaN          NaN   
                       17                     0.009231          NaN   
                       18                     0.009231     0.016923   
                       19                     0.120000          NaN   
                       20                          NaN          NaN   
                       21                          NaN          NaN   
                       22                          NaN     0.016923   
                       24                          NaN          NaN   
                       26                          NaN          NaN   
                       29                          NaN          NaN   
                       36                     0.060000          NaN   
                       39                          NaN          NaN   
                       42                          NaN          NaN   

                             mixedBroadleafandNeedleleafForests  sparseForests  
comName                week                                                     
Canada Jay             1                               0.067692            NaN  
                       10                                   NaN            NaN  
                       11                                   NaN            NaN  
                       13                                   NaN       0.181538  
                       40                                   NaN            NaN  
                       49                              0.067692            NaN  
                     

In [ ]:
##TODO #65 using historic habitat preference data, forecast the seasonal habitat preference for each species of bird